<a href="https://colab.research.google.com/github/FelixMaturano/C---TODO/blob/master/02_pytorch_nn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/sensioai/blog/blob/master/028_pytorch_nn/pytorch_nn.ipynb)

# Pytorch - Redes Neuronales

En el post [anterior](https://sensioai.com/blog/027_pytorch_intro) hicimos una introducción al framework de `redes neuronales` `Pytorch`. Hablamos de sus tres elementos fundamentales: el objeto `tensor` (similar al `array` de `NumPy`) `autograd` (que nos permite calcular derivadas de manera automáticas) y el soporte GPU. En este post vamos a entrar en detalle en la  funcionalidad que nos ofrece la librería para diseñar redes neuronales de manera flexible.

In [ ]:
import torch

## Modelos secuenciales

La forma más sencilla de definir una `red neuronal` en `Pytorch` es utilizando la clase `Sequentail`. Esta clase nos permite definir una secuencia de capas, que se aplicarán de manera secuencial (las salidas de una capa serán la entrada de la siguiente). Ésto ya lo conocemos de posts anteriores, ya que es la forma ideal de definir un `Perceptrón Multicapa`.

In [ ]:
# D_in, H, D_out = 784, 100, 10

# model = torch.nn.Sequential(
#     torch.nn.Linear(D_in, H),
#     torch.nn.ReLU(),
#     torch.nn.Linear(H, D_out),
# )

D_in, H1, H2, D_out = 784, 100, 50, 10
model = torch.nn.Sequential(
    torch.nn.Linear(D_in, H1),
    torch.nn.ReLU(),
    torch.nn.Linear(H1, H2),
    torch.nn.ReLU(),
    torch.nn.Linear(H2, D_out),
)


El modelo anterior es un `MLP` con 784 entradas, 100 neuronas en la capa oculta y 10 salidas. Podemos usar este modelo para hacer un clasificador de imágenes con el dataset MNIST. Pero primero, vamos a ver como podemos calcular las salidas del modelo a partir de unas entradas de ejemplo.

In [ ]:
outputs = model(torch.randn(600, 784))
outputs.shape

torch.Size([600, 10])

In [ ]:
print(outputs[0][:])

tensor([ 0.0792,  0.0338,  0.0941, -0.1050,  0.0973,  0.0971,  0.1367, -0.0140,
         0.0007, -0.1859], grad_fn=<SliceBackward0>)


Como puedes ver, simplemente le pasamos los inputs al modelo (llamándolo como una función). En este caso, usamos un tensor con 64 vectores de 784 valores. Es importante remarcar que los modelos de `Pytorch` (por lo general) siempre esperan que la primera dimensión sea la dimensión *batch*. Si queremos entrenar esta red en una GPU, es tan sencillo como

In [ ]:
model

Sequential(
  (0): Linear(in_features=784, out_features=100, bias=True)
  (1): ReLU()
  (2): Linear(in_features=100, out_features=50, bias=True)
  (3): ReLU()
  (4): Linear(in_features=50, out_features=10, bias=True)
)

In [ ]:
model.to("cuda")

Sequential(
  (0): Linear(in_features=784, out_features=100, bias=True)
  (1): ReLU()
  (2): Linear(in_features=100, out_features=50, bias=True)
  (3): ReLU()
  (4): Linear(in_features=50, out_features=10, bias=True)
)

Vamos a ver ahora como entrenar este modelo con el dataset MNIST.

In [ ]:
from sklearn.datasets import fetch_openml

# descarga datos

mnist = fetch_openml('mnist_784', version=1)
X, Y = mnist["data"], mnist["target"]

X.shape, Y.shape

((70000, 784), (70000,))

In [ ]:
# import numpy as np

# normalización y split
import numpy as np
x_2=np.array(X)
y_2=np.array(Y)

# normalización y split

X_train =x_2[:60000] / 255.
X_test =x_2[60000:] / 255.
y_train = y_2[:60000].astype(np.int32)
y_test = y_2[60000:].astype(np.int32)



# X_train, X_test, y_train, y_test = X[:60000] / 255., X[60000:] / 255., Y[:60000].astype(np.float32), Y[60000:].astype(np.float32)

In [ ]:
# función de pérdida y derivada

def softmax(x):
    return torch.exp(x) / torch.exp(x).sum(axis=-1,keepdims=True)

def cross_entropy(output, target):
    logits = output[torch.arange(len(output)), target]
    loss = - logits + torch.log(torch.sum(torch.exp(output), axis=-1))
    loss = loss.mean()
    return loss

In [ ]:
# X_train

In [ ]:
torch.cuda.is_available()

True

In [ ]:
print(X)

       pixel1  pixel2  pixel3  pixel4  pixel5  pixel6  pixel7  pixel8  pixel9  \
0           0       0       0       0       0       0       0       0       0   
1           0       0       0       0       0       0       0       0       0   
2           0       0       0       0       0       0       0       0       0   
3           0       0       0       0       0       0       0       0       0   
4           0       0       0       0       0       0       0       0       0   
...       ...     ...     ...     ...     ...     ...     ...     ...     ...   
69995       0       0       0       0       0       0       0       0       0   
69996       0       0       0       0       0       0       0       0       0   
69997       0       0       0       0       0       0       0       0       0   
69998       0       0       0       0       0       0       0       0       0   
69999       0       0       0       0       0       0       0       0       0   

       pixel10  ...  pixel7

In [ ]:
# convertimos datos a tensores y copiamos en gpu

X_t = torch.from_numpy(X_train).float().cuda()
Y_t = torch.from_numpy(y_train).long().cuda()

# bucle entrenamiento
epochs = 350
lr = 0.8
log_each = 10
l = []
for e in range(1, epochs + 1):

    # forward
    y_pred = model(X_t)

    # loss
    loss = cross_entropy(y_pred, Y_t)
    l.append(loss.item())

    # ponemos a cero los gradientes
    model.zero_grad()

    # Backprop (calculamos todos los gradientes automáticamente)
    loss.backward()

    # update de los pesos
    with torch.no_grad():
        for param in model.parameters():
            param -= lr * param.grad

    if not e % log_each:
        print(f"Epoch {e}/{epochs} Loss {np.mean(l):.5f}")

Epoch 10/350 Loss 2.06514
Epoch 20/350 Loss 2.06357
Epoch 30/350 Loss 1.93566
Epoch 40/350 Loss 1.73623
Epoch 50/350 Loss 1.54942
Epoch 60/350 Loss 1.39333
Epoch 70/350 Loss 1.26210
Epoch 80/350 Loss 1.15828
Epoch 90/350 Loss 1.06748
Epoch 100/350 Loss 0.99519
Epoch 110/350 Loss 0.93133
Epoch 120/350 Loss 0.88082
Epoch 130/350 Loss 0.83544
Epoch 140/350 Loss 0.79244
Epoch 150/350 Loss 0.75640
Epoch 160/350 Loss 0.72562
Epoch 170/350 Loss 0.69497
Epoch 180/350 Loss 0.67293
Epoch 190/350 Loss 0.64872
Epoch 200/350 Loss 0.62534
Epoch 210/350 Loss 0.60378
Epoch 220/350 Loss 0.58391
Epoch 230/350 Loss 0.56655
Epoch 240/350 Loss 0.55265
Epoch 250/350 Loss 0.53667
Epoch 260/350 Loss 0.52166
Epoch 270/350 Loss 0.50758
Epoch 280/350 Loss 0.49434
Epoch 290/350 Loss 0.48188
Epoch 300/350 Loss 0.47011
Epoch 310/350 Loss 0.45901
Epoch 320/350 Loss 0.44882
Epoch 330/350 Loss 0.47321
Epoch 340/350 Loss 0.49810
Epoch 350/350 Loss 0.50058


Como puedes observar en el ejemplo, podemos calcular la salida del modelo con una simple línea. Luego calculamos la función de pérdida, y llamando a la función `backward` `Pytorch` se encarga de calcular las derivadas de la misma con respecto a todos los parámetros del modelo automáticamente (si no queremos acumular estos gradientes, nos aseguramos de llamar a la función `zero_grad` para ponerlos a cero antes de calcularlos). Por útlimo, podemos iterar por los parámetros del modelo aplicando la regla de actualización deseada (en este caso usamos `descenso por gradiente`).

In [ ]:
from sklearn.metrics import accuracy_score

def evaluate(x):
    model.eval()
    y_pred = model(x)
    y_probas = softmax(y_pred)
    return torch.argmax(y_probas, axis=1)

y_pred = evaluate(torch.from_numpy(X_test).float().cuda())
accuracy_score(y_test, y_pred.cpu().numpy())

0.8948

Existen algunos tipos de capas que se comportan diferente en función de si estamos entrenando la red o usándola para generar predicciones. Podemos controlar el modo en el que queremos que esté nuestra red con las funciones `train` y `eval`.

## Optimizadores y Funciones de pérdida

En el ejemplo anterior hemos calculado la función de pérdida y aplicado la regla de optimización de forma manual. Sin embargo, `Pytorch` nos ofrece funcionalidad que nos abstrae estos cálculos ofreciendo además flexibilidad para aplicar diferentes funciones de pérdida o algoritmos de optimización de manera sencilla. Podemos encontrar diferentes funciones de pérdida ya implementadas en el paquete `torch.nn`.

In [ ]:
criterion = torch.nn.CrossEntropyLoss()

Mientras que los optimizadores se encuentran en el paquete `torch.optim`

In [ ]:
optimizer = torch.optim.SGD(model.parameters(), lr=0.8)

Puedes ver la lista completa de funciones de pérdida y optimizadores disponibles en la [documentación](https://pytorch.org/docs/stable/index.html), aunque como ya has visto siempre puedes definir los tuyos propios fácilmente.

Una vez definidos estos dos objetos, nuestro bucle de entrenamiento se simplifica considerablemente.

In [ ]:
D_in, H, D_out = 784, 100, 10

model = torch.nn.Sequential(
    torch.nn.Linear(D_in, H),
    torch.nn.ReLU(),
    torch.nn.Linear(H, D_out),
).to("cuda")

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.8)

epochs = 1500
log_each = 10
l = []
model.train()
for e in range(1, epochs+1):

    # forward
    y_pred = model(X_t)

    # loss
    loss = criterion(y_pred, Y_t)
    l.append(loss.item())

    # ponemos a cero los gradientes
    optimizer.zero_grad()

    # Backprop (calculamos todos los gradientes automáticamente)
    loss.backward()

    # update de los pesos
    optimizer.step()

    if not e % log_each:
        print(f"Epoch {e}/{epochs} Loss {np.mean(l):.5f}")

y_pred = evaluate(torch.from_numpy(X_test).float().cuda())
accuracy_score(y_test, y_pred.cpu().numpy())

Epoch 10/1500 Loss 1.74943
Epoch 20/1500 Loss 1.46122
Epoch 30/1500 Loss 1.20877
Epoch 40/1500 Loss 1.03928
Epoch 50/1500 Loss 0.91407
Epoch 60/1500 Loss 0.81729
Epoch 70/1500 Loss 0.74464
Epoch 80/1500 Loss 0.68873
Epoch 90/1500 Loss 0.64303
Epoch 100/1500 Loss 0.60528
Epoch 110/1500 Loss 0.57375
Epoch 120/1500 Loss 0.54602
Epoch 130/1500 Loss 0.52166
Epoch 140/1500 Loss 0.50021
Epoch 150/1500 Loss 0.48115
Epoch 160/1500 Loss 0.46407
Epoch 170/1500 Loss 0.44864
Epoch 180/1500 Loss 0.43463
Epoch 190/1500 Loss 0.42181
Epoch 200/1500 Loss 0.41003
Epoch 210/1500 Loss 0.39915
Epoch 220/1500 Loss 0.38906
Epoch 230/1500 Loss 0.37967
Epoch 240/1500 Loss 0.37089
Epoch 250/1500 Loss 0.36267
Epoch 260/1500 Loss 0.35493
Epoch 270/1500 Loss 0.34764
Epoch 280/1500 Loss 0.34076
Epoch 290/1500 Loss 0.33424
Epoch 300/1500 Loss 0.32805
Epoch 310/1500 Loss 0.32217
Epoch 320/1500 Loss 0.31656
Epoch 330/1500 Loss 0.31122
Epoch 340/1500 Loss 0.30611
Epoch 350/1500 Loss 0.30122
Epoch 360/1500 Loss 0.29654
E

0.976

## Modelos custom

Si bien en muchos casos definir una `red neuronal` como una secuencia de capas es suficiente, en otros casos será un factor limitante. Un ejemplo son las redes residuales, en las que no sólo utilizamos la salida de una capa para alimentar la siguiente si no que, además, le sumamos su propia entrada. Este tipo de arquitectura no puede ser definida con la clase `Sequential`, y para ello necesitamos hacer un modelo *customizado*. Para ello, `Pytroch` nos ofrece la siguiente sintaxis.

In [ ]:
# creamos una clase que hereda de `torch.nn.Module`

class ModeloPersonalizado(torch.nn.Module):

    # constructor
    def __init__(self, D_in, H, D_out):

        # llamamos al constructor de la clase madre
        super(ModeloPersonalizado, self).__init__()

        # definimos nuestras capas
        self.fc1 = torch.nn.Linear(D_in, H)
        self.relu = torch.nn.ReLU()
        self.fc2 = torch.nn.Linear(H, D_out)

    # lógica para calcular las salidas de la red
    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

En primer lugar, necesitamos definir una nueva clase que herede de la clase `torch.nn.Module`. Esta clase madre aportará toda la funcionalidad esencial que necesita una `red neuronal` (soporte GPU, iterar por sus parámeteros, etc). Luego, en esta clase necesitamos definir mínimos dos funciones:

- `init`: en el constructor llamaremos al constructor de la clase madre y después definiremos todas las capas que querramos usar en la red.
- `forward`: en esta función definimos toda la lógica que aplicaremos desde que recibimos los inputs hasta que devolvemos los outputs.

En el ejemplo anterior simplemente hemos replicado la misma red (puedes conseguir el mismo efecto usando la clase `Sequential`).

In [ ]:
model = ModeloPersonalizado(784, 100, 10)
# Codigo para saber si el modelo esta votando los datos en las cantidades correctas
x_prueba = torch.randn(500, 784)
print(x_prueba)
outputs = model(x_prueba)
outputs.shape

tensor([[-0.9555, -0.4134,  0.0208,  ...,  1.0999,  0.6025,  1.9015],
        [-1.0949, -2.8468,  0.9335,  ...,  0.5056, -1.8322,  0.4758],
        [ 0.1603,  0.0248,  0.2974,  ..., -0.2047,  2.2994, -1.5774],
        ...,
        [ 0.1713, -0.4712,  0.4067,  ..., -0.1814,  0.8939,  1.0035],
        [-1.7229, -0.7686,  0.3269,  ..., -0.4310,  0.3240, -3.4103],
        [ 0.5819,  0.1190,  0.3799,  ..., -1.5761,  0.7148, -1.6947]])


torch.Size([500, 10])

Ahora, podemos entrenar nuestra red de la misma forma que lo hemos hecho anteriormente.

In [ ]:
model.to("cuda")
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.8)

epochs = 100
log_each = 10
l = []
model.train()
for e in range(1, epochs+1):

    # forward
    y_pred = model(X_t)

    # loss
    loss = criterion(y_pred, Y_t)
    l.append(loss.item())

    # ponemos a cero los gradientes
    optimizer.zero_grad()

    # Backprop (calculamos todos los gradientes automáticamente)
    loss.backward()

    # update de los pesos
    optimizer.step()

    if not e % log_each:
        print(f"Epoch {e}/{epochs} Loss {np.mean(l):.5f}")

y_pred = evaluate(torch.from_numpy(X_test).float().cuda())
accuracy_score(y_test, y_pred.cpu().numpy())

Epoch 10/100 Loss 1.77221
Epoch 20/100 Loss 1.53883
Epoch 30/100 Loss 1.26340
Epoch 40/100 Loss 1.07572
Epoch 50/100 Loss 0.94111
Epoch 60/100 Loss 0.84378
Epoch 70/100 Loss 0.76844
Epoch 80/100 Loss 0.70951
Epoch 90/100 Loss 0.66208
Epoch 100/100 Loss 0.62237


0.928

Aquí puedes ver otro ejemplo de como definir un `MLP` con conexiones residuales, algo que no podemos hacer simplemente usando un modelo secuencial.

In [ ]:
class ModelCustom2(torch.nn.Module):

    def __init__(self, D_in, H, D_out):
        super(ModelCustom2, self).__init__()
        self.fc1 = torch.nn.Linear(D_in, H)
        self.relu = torch.nn.ReLU()
        self.fc2 = torch.nn.Linear(H, D_out)

    def forward(self, x):
        x1 = self.fc1(x)
        x = self.relu(x1)
        x = self.fc2(x + x1)
        return x

In [ ]:
model = ModelCustom2(784, 100, 10).to("cuda")
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.2)

epochs = 100
log_each = 10
l = []
model.train()
for e in range(1, epochs+1):

    # forward
    y_pred = model(X_t)

    # loss
    loss = criterion(y_pred, Y_t)
    l.append(loss.item())

    # ponemos a cero los gradientes
    optimizer.zero_grad()

    # Backprop (calculamos todos los gradientes automáticamente)
    loss.backward()

    # update de los pesos
    optimizer.step()

    if not e % log_each:
        print(f"Epoch {e}/{epochs} Loss {np.mean(l):.5f}")

y_pred = evaluate(torch.from_numpy(X_test).float().cuda())
accuracy_score(y_test, y_pred.cpu().numpy())

Epoch 10/100 Loss 1.61990
Epoch 20/100 Loss 1.18053
Epoch 30/100 Loss 0.96668
Epoch 40/100 Loss 0.87062
Epoch 50/100 Loss 0.78109
Epoch 60/100 Loss 0.71820
Epoch 70/100 Loss 0.67372
Epoch 80/100 Loss 0.63546
Epoch 90/100 Loss 0.60382
Epoch 100/100 Loss 0.57755


0.9108

De esta manera, tenemos mucha flexibilidad para definir nuestras redes.

## Accediendo a las capas de una red

En ocasiones queremos acceder a una capa en particular de nuestra red. Para ello, podemos acceder utilizando su nombre.

In [ ]:
model

ModelCustom2(
  (fc1): Linear(in_features=784, out_features=100, bias=True)
  (relu): ReLU()
  (fc2): Linear(in_features=100, out_features=10, bias=True)
)

In [ ]:
model.fc1

Linear(in_features=784, out_features=100, bias=True)

También podemos acceder directamente a los tensores que contienen los parámetros con las propiedades adecuadas

In [ ]:
model.fc1.weight

Parameter containing:
tensor([[ 0.0286,  0.0047, -0.0349,  ..., -0.0220,  0.0276, -0.0311],
        [-0.0034,  0.0025, -0.0113,  ..., -0.0299,  0.0035, -0.0332],
        [ 0.0155,  0.0121,  0.0241,  ...,  0.0043, -0.0202,  0.0162],
        ...,
        [-0.0061,  0.0156,  0.0068,  ..., -0.0301,  0.0255,  0.0275],
        [-0.0247, -0.0235, -0.0240,  ...,  0.0296, -0.0268,  0.0025],
        [ 0.0307, -0.0045,  0.0036,  ...,  0.0007, -0.0042,  0.0281]],
       device='cuda:0', requires_grad=True)

In [ ]:
model.fc1.bias

Parameter containing:
tensor([ 0.0132, -0.0469,  0.0579, -0.0061,  0.0288,  0.0308,  0.0268, -0.0632,
        -0.0221,  0.0825,  0.0223,  0.0274,  0.0107, -0.0335, -0.0315, -0.0444,
         0.0047,  0.0432, -0.0019, -0.0073, -0.0321,  0.0277, -0.0113,  0.0872,
         0.0851,  0.0554,  0.0549, -0.0228, -0.0323, -0.0163,  0.0045,  0.0323,
        -0.0176,  0.0337, -0.0084,  0.0364,  0.0456, -0.0343, -0.0397,  0.0622,
         0.0294,  0.0085, -0.0032, -0.0219,  0.0458,  0.0049,  0.0190,  0.0207,
         0.0243, -0.0145, -0.0561,  0.0012,  0.0216, -0.0144,  0.0462, -0.0336,
         0.0759, -0.0034,  0.0003,  0.0103, -0.0055,  0.0236, -0.0201,  0.0549,
         0.0065, -0.0110,  0.0447,  0.0171,  0.0103,  0.0581, -0.0040, -0.0082,
         0.0332,  0.0201,  0.0597,  0.0298,  0.0566, -0.0082,  0.0096, -0.0394,
         0.0169,  0.0231, -0.0275, -0.0226, -0.0060, -0.0241,  0.0432,  0.0602,
         0.0305, -0.0648, -0.0134,  0.0300,  0.0254,  0.0121,  0.0283,  0.0126,
        -0.0269,  

Es posible sobreescribir una capa de la siguiente manera

In [ ]:
model.fc2 = torch.nn.Linear(100, 1)

model

ModelCustom2(
  (fc1): Linear(in_features=784, out_features=100, bias=True)
  (relu): ReLU()
  (fc2): Linear(in_features=100, out_features=1, bias=True)
)

Ahora, la capa final de nuestra red tiene solo una salida. Esta nueva capa ha sido inicializada de manera aleatoria, por lo que esta nueva red no nos va a servir de mucho. Sin embargo, podríamos volver a entrenar esta red en otro problema en el que solo necesitemos una salida aprovechando los pesos que ya hemos entrenado anteriormente con el dataset MNIST. Esto es la base del *transfer learning*, una técnica que utilizaremos muchísimo más adelante y la cual explicaremos en detalle.

A continuación encontrarás varios trucos a la hora de crear redes neuronales a partir de otras que te pueden resultar útiles.

In [ ]:
# obtener una lista con las capas de una red

list(model.children())

[Linear(in_features=784, out_features=100, bias=True),
 ReLU(),
 Linear(in_features=100, out_features=1, bias=True)]

In [ ]:
# crear nueva red a partir de la lista (excluyendo las útlimas dos capa)

new_model = torch.nn.Sequential(*list(model.children())[:-2])
new_model

Sequential(
  (0): Linear(in_features=784, out_features=100, bias=True)
)

In [ ]:
# crear nueva red a partir de la lista (excluyendo las útlima capa)

new_model = torch.nn.ModuleList(list(model.children())[:-1])
new_model

ModuleList(
  (0): Linear(in_features=784, out_features=100, bias=True)
  (1): ReLU()
)

## Resumen

En este post hemos visto la funcionalidad que `Pytorch` nos ofrece a la hora de definir y entrenar nuestras `redes neuronales`. El paquete `torch.nn` contiene todo lo necesario para diseñar nuestros modelos, ya sea de manera secuencial o con una clase *custom* para arquitecturas más complicadas. También nos da muchas funciones de pérdida que podemos usar directamente para entrenar las redes. Te recomiendo encarecidamente que le eches un vistazo a la [documentación](https://pytorch.org/docs/stable/nn.html) par hacerte una idea de todo lo que puedes hacer. También hemos visto como el paquete `torch.optim` nos oferece algoritmos de optimización que también nos hacen la vida más fácil a la hora de entrenar nuestras redes.